# Tugas

## Tugas 1

Terdapat dataset mushroom. Berdasarkan dataset yang tersebut, bandingkan peforma antara algoritma Decision Tree dan RandomForest. Gunakan tunning hyperparameter untuk mendapatkan parameter dan akurasi yang terbaik.

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
mushroom_data = pd.read_csv('mushrooms.csv')

# Pisahkan fitur dan label
X = mushroom_data.drop('class', axis=1)
y = mushroom_data['class']

# Encode data kategorikal menjadi numerik
X = pd.get_dummies(X)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Tuning hyperparameter untuk Decision Tree
dt_params = {'max_depth': [5, 10, 15, 20], 'min_samples_split': [2, 5, 10]}
dt = GridSearchCV(DecisionTreeClassifier(random_state=42), dt_params, cv=5)
dt.fit(X_train, y_train)
dt_best = dt.best_estimator_

# Tuning hyperparameter untuk Random Forest
rf_params = {'n_estimators': [50, 100], 'max_depth': [10, 20, None]}
rf = GridSearchCV(RandomForestClassifier(random_state=42), rf_params, cv=5)
rf.fit(X_train, y_train)
rf_best = rf.best_estimator_

# Evaluasi performa
y_pred_dt = dt_best.predict(X_test)
y_pred_rf = rf_best.predict(X_test)

print("Decision Tree Accuracy:", accuracy_score(y_test, y_pred_dt))
print("Decision Tree Report:\n", classification_report(y_test, y_pred_dt))

print("Random Forest Accuracy:", accuracy_score(y_test, y_pred_rf))
print("Random Forest Report:\n", classification_report(y_test, y_pred_rf))


Decision Tree Accuracy: 1.0
Decision Tree Report:
               precision    recall  f1-score   support

           e       1.00      1.00      1.00      1052
           p       1.00      1.00      1.00       979

    accuracy                           1.00      2031
   macro avg       1.00      1.00      1.00      2031
weighted avg       1.00      1.00      1.00      2031

Random Forest Accuracy: 1.0
Random Forest Report:
               precision    recall  f1-score   support

           e       1.00      1.00      1.00      1052
           p       1.00      1.00      1.00       979

    accuracy                           1.00      2031
   macro avg       1.00      1.00      1.00      2031
weighted avg       1.00      1.00      1.00      2031



### Penjelasan:
Membandingkan performa Decision Tree dan Random Forest dengan tuning hyperparameter menggunakan GridSearchCV.

- Tujuan: Membandingkan akurasi dan performa dari dua algoritma:
    Decision Tree: Model berbasis pohon keputusan yang memilih fitur secara rekursif untuk membagi data berdasarkan impuritas (misalnya Gini atau Entropy).
    Random Forest: Kumpulan banyak pohon keputusan dengan teknik bagging untuk meningkatkan generalisasi dan mengurangi overfitting.

- Langkah Tuning Hyperparameter:
    Decision Tree: Dicari kombinasi terbaik untuk max_depth (kedalaman maksimum) dan min_samples_split (minimal sampel agar terjadi pembagian).
    Random Forest: Dicari kombinasi terbaik untuk jumlah pohon (n_estimators) dan kedalaman pohon (max_depth).

- Evaluasi:
    Akurasi dan laporan klasifikasi ditampilkan untuk membandingkan performa kedua model.
    Random Forest biasanya lebih akurat daripada Decision Tree karena menggunakan banyak pohon yang mengurangi risiko overfitting.

## Tugas 2

Bandingkan performa antara Decision Tree dan AdaBoost menggunakan dataset mushroom, dengan tuning hyperparameter.

In [3]:
from sklearn.ensemble import AdaBoostClassifier

# Tuning hyperparameter untuk AdaBoost
ab_params = {'n_estimators': [50, 100], 'learning_rate': [0.01, 0.1, 1.0]}
ab = GridSearchCV(AdaBoostClassifier(random_state=42), ab_params, cv=5)
ab.fit(X_train, y_train)
ab_best = ab.best_estimator_

# Evaluasi performa
y_pred_ab = ab_best.predict(X_test)

print("AdaBoost Accuracy:", accuracy_score(y_test, y_pred_ab))
print("AdaBoost Report:\n", classification_report(y_test, y_pred_ab))


AdaBoost Accuracy: 1.0
AdaBoost Report:
               precision    recall  f1-score   support

           e       1.00      1.00      1.00      1052
           p       1.00      1.00      1.00       979

    accuracy                           1.00      2031
   macro avg       1.00      1.00      1.00      2031
weighted avg       1.00      1.00      1.00      2031



### Penjelasan
Membandingkan performa Decision Tree dan AdaBoost dengan metode yang sama.

- Tujuan: Membandingkan performa antara:
    Decision Tree: Model sederhana dan cepat, tetapi rentan terhadap overfitting.
    AdaBoost: Teknik boosting yang membangun model secara iteratif, menekankan pada sampel yang sulit diklasifikasikan dalam iterasi sebelumnya.

- Langkah Tuning Hyperparameter:
    Decision Tree: Parameter sama seperti pada Tugas 1.
    AdaBoost: Hyperparameter yang dicari:
        n_estimators: Jumlah estimator (pohon) dalam ensemble.
        learning_rate: Mengontrol kontribusi masing-masing pohon.

- Evaluasi:
Model AdaBoost sering memberikan akurasi lebih tinggi karena memperkuat sampel yang sulit diklasifikasikan.


## Tugas 3

Buat ensemble voting menggunakan Logistic Regression, SVM dengan kernel polynomial, dan Decision Tree menggunakan dataset diabetes.

In [4]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler

# Load dataset diabetes
diabetes = pd.read_csv('diabetes.csv')

# Pisahkan fitur dan label
X = diabetes.drop('Outcome', axis=1)
y = diabetes['Outcome']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Standarisasi fitur
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Definisikan model
logreg = LogisticRegression(random_state=42)
svm = SVC(kernel='poly', probability=True, random_state=42)
dt = DecisionTreeClassifier(random_state=42)

# Ensemble voting
voting_clf = VotingClassifier(estimators=[('lr', logreg), ('svc', svm), ('dt', dt)], voting='soft')
voting_clf.fit(X_train, y_train)

# Evaluasi performa
y_pred_voting = voting_clf.predict(X_test)
print("Voting Classifier Accuracy:", accuracy_score(y_test, y_pred_voting))
print("Classification Report:\n", classification_report(y_test, y_pred_voting))


Voting Classifier Accuracy: 0.7760416666666666
Classification Report:
               precision    recall  f1-score   support

           0       0.79      0.89      0.84       125
           1       0.73      0.57      0.64        67

    accuracy                           0.78       192
   macro avg       0.76      0.73      0.74       192
weighted avg       0.77      0.78      0.77       192



### Penjelasan
Menggabungkan Logistic Regression, SVM dengan kernel polynomial, dan Decision Tree dalam ensemble voting. Standarisasi digunakan untuk memastikan SVM bekerja optimal.

- Tujuan: Membuat ensemble voting dengan tiga algoritma:
    Logistic Regression: Model sederhana dan linier, cocok untuk klasifikasi biner.
    SVM dengan kernel polynomial: Mampu menangani data non-linier dengan proyeksi ke dimensi lebih tinggi.
    Decision Tree: Model berbasis pohon yang menangkap hubungan non-linier.

- Langkah Tuning Hyperparameter:
    Hyperparameter dari tiap model bisa dieksplorasi (seperti kernel pada SVM, atau kedalaman pada Decision Tree).
    VotingClassifier: Voting bisa diatur dalam dua mode:
        Hard Voting: Berdasarkan mayoritas prediksi.
        Soft Voting: Berdasarkan probabilitas dari masing-masing model.

- Evaluasi:
    Standarisasi Data: Dilakukan karena SVM sangat sensitif terhadap skala data.
    Akurasi dan Laporan Klasifikasi: Digunakan untuk mengukur performa ensemble dan mengevaluasi model mana yang memiliki performa terbaik.